In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from glob import glob
from pathlib import Path
from openpiv import tools

from utils.piv.calculate_field import calculate_field_from_multiple_frames
from utils.piv.analysis import create_mask, calculate_center, plot_with_background

# Cálculo del campo de velocidades.

In [ ]:
# Parametros de preprocesamiento para cada video.
# x e y: Offset del centro en x e y.
# r: Radio del circulo que enmascara todo.
# start: Segundo en el que iniciar a extraer.
# total_secs: Segundos totales a extraer.
# scale: Conversion de px a cm.
# Crear una fila por cada nombre de archivo como en el archivo de ejemplo abajo.

df = pd.read_csv("preproc_data_1104.csv", sep=",", engine='python').set_index("filename")

In [ ]:
files = list(map(Path, glob("./proc_images/*")))

for f in files:
    print(f'Processing {f.name[:-4]}')
    
    skip_frames = 15 # Fotogramas a saltear del inicio por el Removedor de Fondos.
    scale_factor = df.loc[f.name].loc['scale']/10 # Pasar de px/cm a px/mm

    # Calcular el campo de velocidades para cada par de fotogramas. Interpola el resultado.
    mean_x, mean_y, mean_u, mean_v = calculate_field_from_multiple_frames(f, 
                                                                        skip_frames=skip_frames, 
                                                                        end_frame=-1,
                                                                        dt=1/30, 
                                                                        searchsize=24, 
                                                                        winsize=8, 
                                                                        overlap=4, 
                                                                        threshold=1, 
                                                                        kernel_size=5, 
                                                                        max_iter=3,
                                                                        scale_factor = scale_factor)

    # Guardar archivo comprimido para evitar volver a calcular el campo.
    np.savez_compressed(file=f'npz/{f.name[:-4]}.npz', x = mean_x, y = mean_y, u = mean_u, v = mean_v)

    # Voy graficando para cada video.
    plt.quiver(mean_x, mean_y, mean_u, mean_v)
    plt.title(f.name[:-4])
    plt.show()

# Cálculo del centro y obtención de perfiles.

## Un ejemplo para un video. Estimar parametros para automatizar.

In [ ]:
# Parametros de preprocesamiento para cada video.
# threshold: Modulo minimo de los vectores a tomar.
# max_r: Radio maximo a tomar.
# min_r: Radio minimo a tomar.

data = np.load(f'npz/fields/2 G50 RC.npz') # Cambiar Path

mean_x = data['x']
mean_y = data['y']
mean_u = data['u']
mean_v = data['v']

background = tools.imread(f'images/2 G50 RC.mp4/00000090.jpg') # Cambiar Path
fig, ax = plot_with_background(background, mean_x, mean_y, mean_u, mean_v)

threshold, min_r, max_r = 110, 20, 80

# Mascara de puntos para calcular el centro.
sel_x, sel_y, sel_u, sel_v = create_mask(mean_x, mean_y, mean_u, mean_v, 
                                         threshold = threshold,
                                         min_r = min_r,
                                         max_r = max_r,
                                         plot_mask = True,
                                         background=background)

# Calculo del centro.
x_c, y_c, x_c_err, y_c_err = calculate_center(mean_x, mean_y, mean_u, mean_v, 
                                              threshold = threshold,
                                              min_r = min_r,
                                              max_r = max_r,
                                              plot_results = True,
                                              background=background)

# Todos los videos:

In [ ]:
# Parametros de preprocesamiento para cada video.
# threshold: Modulo minimo de los vectores a tomar.
# max_r: Radio maximo a tomar.
# min_r: Radio minimo a tomar.
# Crear una fila por cada nombre de archivo como en el archivo de ejemplo abajo.
df = pd.read_csv('proc_fields.csv', engine='python', sep=',').set_index('filename')

In [ ]:
files = list(map(Path, glob('npz/fields/*')))

for f in files:
    data = np.load(f'npz/fields/{f.name}')

    # Levantar datos del campo.
    mean_x = data['x']
    mean_y = data['y']
    mean_u = data['u']
    mean_v = data['v']

    # Calculo del centro.
    x_c, y_c, x_c_err, y_c_err = calculate_center(mean_x, mean_y, mean_u, mean_v, 
                                                threshold = df.loc[f.name].loc['threshold'],
                                                min_r = df.loc[f.name].loc['min_r'],
                                                max_r = df.loc[f.name].loc['max_r'],
                                                plot_results = False)

    # Radio en mm del recipiente grande.
    max_r = 98.5

    # Centrar las matrices de x e y,
    centered_x = mean_x.flatten() - x_c
    centered_y = mean_y.flatten() - y_c

    # Calcular el centro, y los valores de theta.
    r = np.sqrt(centered_x**2 + centered_y**2)
    cos_theta = centered_x / r
    sin_theta = centered_y / r

    # Calculo componentes del campo
    Ur = cos_theta * mean_u.flatten() + sin_theta * mean_v.flatten()
    Utheta = abs(-sin_theta * mean_u.flatten() + cos_theta * mean_v.flatten())

    # Me quedo con los de radio menor al r_max.
    masked_Ur = Ur[r < max_r]
    masked_Utheta = Utheta[r < max_r]
    masked_r = r[r < max_r]

    masked_Ur = masked_Ur[~np.isnan(masked_Utheta)]
    masked_r = masked_r[~np.isnan(masked_Utheta)]
    masked_Utheta = masked_Utheta[~np.isnan(masked_Utheta)]

    np.savez_compressed(f'npz/processed/{f.name}', r = masked_r, Ur=masked_Ur, Utheta=masked_Utheta)